In [1]:
setwd("C:/Users/User/Documents")

In [2]:
pacotes <- c("tidyverse","tidyquant","reticulate","BatchGetSymbols")
for(i in pacotes){
  suppressPackageStartupMessages(library(i,character.only = T))
}

Warning message:
"package 'BatchGetSymbols' was built under R version 4.0.3"


In [39]:
#Stocks to be ranked
si <- import("yahoo_fin.stock_info","si")
max_acoes <- 100
tickers <- GetIbovStocks()$tickers[1:max_acoes]
tickers <- paste0(tickers,".SA")

In [40]:
#Downloading the data
b <- replicate(length(tickers),list())
for(i in 1:length(b)){
  valuation <- si$get_stats_valuation(tickers[i])[,1:2]
  dados <- c("Beta (5Y Monthly)","Profit Margin","Return on Equity (ttm)",
             "Return on Assets (ttm)","Quarterly Revenue Growth (yoy)")
  stats <- data.frame(x = dados, y = rep(NaN,length(dados)))
  try({stats <- si$get_stats(tickers[i])
  stats <- stats[which(stats[,1] %in% dados),]
  stats$Value <- as.numeric(gsub("%","",stats$Value,fixed = T))
  stats$Value <- as.numeric(stats$Value)},silent = T)
  colnames(stats) <- colnames(valuation)
  b[[i]] <- rbind(stats,valuation)
}

ERROR: Error in py_call_impl(callable, dots$args, dots$keywords): IndexError: list index out of range

Detailed traceback: 
  File "C:\Users\User\anaconda3\lib\site-packages\yahoo_fin\stock_info.py", line 267, in get_stats_valuation
    table = tables[0].reset_index(drop = True)



In [41]:
#Statistics in the rank
statistics <- c("Trailing P/E","PEG Ratio (5 yr expected) 1",
                "Enterprise Value/Revenue 3","Enterprise Value/EBITDA 6",
                "Price/Book (mrq)","Price/Sales (ttm)",
                "Beta (5Y Monthly)","Profit Margin","Return on Equity (ttm)",
                "Return on Assets (ttm)","Quarterly Revenue Growth (yoy)")
maior <- c("PEG Ratio (5 yr expected) 1","Profit Margin","Return on Equity (ttm)",
           "Return on Assets (ttm)","Quarterly Revenue Growth (yoy)")
menor <- statistics[!(statistics %in% maior)]

In [42]:
#Stocks Data
tot <- matrix(nrow = length(tickers), ncol = length(statistics))
try(for(i in 1:nrow(tot)){
  for(j in 1:ncol(tot)){
    tot[i,j] <- b[[i]][,2][grepl(statistics[j],b[[i]][,1],fixed = T)] %>%
      as.numeric
    if(is.nan(tot[i,j])){
      tot[i,j] <- NA
    }
  }
}, silent = T)
colnames(tot) <- statistics
rownames(tot) <- tickers
na <- apply(tot,1,function(x)!all(is.na(x)))
tot <- tot[na,]

Warning message in function_list[[k]](value):
"NAs introduzidos por coerção"


In [43]:
#Dividing the data by statistics that need to be maximized and that need to be minimized
div <- 100*ceiling(max(abs(tot),na.rm = T)/100)
tot1 <- matrix(nrow = nrow(tot), ncol = ncol(tot))
colnames(tot1) <- colnames(tot)
rownames(tot1) <- rownames(tot)
for(i in 1:nrow(tot1)){
  for(j in 1:ncol(tot1)){
    if(colnames(tot1)[j] %in% maior){
      tot1[i,j] <- 1/(1+tot[i,j]/div)
    }else{
      tot1[i,j] <- tot[i,j]
    }
  }
}
tot1[,menor] <- ifelse(tot1[,menor] < 0, NA, tot1[,menor])

In [44]:
ranking <- apply(tot1,2,rank, ties.method = "min", na.last = "keep")
na <- matrix(nrow = nrow(ranking), ncol = ncol(ranking))
for(i in 1:nrow(ranking)){
  for(j in 1:ncol(ranking)){
    na[i,j] <- is.na(ranking[i,j])
    if(is.na(ranking[i,j])){
      ranking[i,j] <- max(ranking[,j],na.rm = T)
    }
  }
}
for(i in 1:ncol(na)){
  ranking[na[,i],i] <- ranking[na[,i],i]+1
}

In [45]:
#Weighted ranking
pesos <- c(1,1,1,1,1,1,1,1,1,1,1)
ranking_f <- matrix(nrow = nrow(ranking), ncol = ncol(ranking))
rownames(ranking_f) <- rownames(ranking)
colnames(ranking_f) <- colnames(ranking)
for(i in 1:ncol(ranking)){
  ranking_f[,i] <- ranking[,i]*pesos[i]
}

In [46]:
#Final rank
infi <- apply(ranking_f,2,function(x)any(!is.infinite(x)))
rank_medio <- apply(ranking_f[,infi],1,mean, na.rm = T)
print(sort(rank_medio),3)

 MRFG3.SA  BEEF3.SA  JBSS3.SA  ENBR3.SA  ELET3.SA  ELET6.SA  CRFB3.SA  CPFE3.SA 
     14.9      16.7      17.3      17.7      18.3      18.7      19.5      20.5 
 CYRE3.SA  EGIE3.SA  PCAR3.SA  EQTL3.SA  VIVT3.SA  GOAU4.SA  BRFS3.SA  ABEV3.SA 
     22.2      22.4      22.6      23.1      23.9      25.5      25.7      27.5 
 SBSP3.SA  ECOR3.SA  HGTX3.SA  PRIO3.SA  CSNA3.SA  CMIG4.SA  CIEL3.SA  GGBR4.SA 
     27.7      28.3      28.3      29.4      29.5      29.8      30.0      30.0 
 MRVE3.SA  VALE3.SA  BRDT3.SA  HYPE3.SA  EZTC3.SA  BBAS3.SA  QUAL3.SA  MULT3.SA 
     30.4      30.6      31.1      31.3      32.2      32.3      32.3      33.2 
 BBSE3.SA  IRBR3.SA  B3SA3.SA  HAPV3.SA  ITSA4.SA SULA11.SA  TIMS3.SA  UGPA3.SA 
     33.4      33.4      33.6      33.8      34.0      35.4      35.7      35.9 
 CCRO3.SA  IGTA3.SA  LAME4.SA  RADL3.SA  YDUQ3.SA  LREN3.SA  USIM5.SA  WEGE3.SA 
     36.1      36.1      36.1      36.1      36.1      36.3      36.3      36.5 
 VVAR3.SA  FLRY3.SA  RAIL3.S

In [47]:
tot

,Trailing P/E,PEG Ratio (5 yr expected) 1,Enterprise Value/Revenue 3,Enterprise Value/EBITDA 6,Price/Book (mrq),Price/Sales (ttm),Beta (5Y Monthly),Profit Margin,Return on Equity (ttm),Return on Assets (ttm),Quarterly Revenue Growth (yoy)
ABEV3.SA,26.31,2.41,3.79,12.37,2.90,4.11,0.77,15.61,12.36,7.81,30.5
AZUL4.SA,16.89,NA,3.93,-3.90,NA,1.63,1.41,-176.69,NA,-5.23,-73.4
B3SA3.SA,29.49,NA,14.01,17.01,4.45,14.92,1.05,49.32,15.04,7.05,49.6
BBAS3.SA,5.80,NA,NA,NA,0.82,1.14,1.36,25.42,14.77,1.03,1.3
BBDC3.SA,9.55,1.23,NA,NA,1.41,1.92,NA,NA,NA,NA,NA
BBDC4.SA,10.61,1.37,NA,NA,1.57,2.14,NA,NA,NA,NA,NA
BBSE3.SA,13.79,NA,16.69,NA,8.92,17.25,0.94,76.97,48.43,25.31,6.5
BEEF3.SA,5.33,NA,0.55,4.96,4.45,0.26,0.85,4.45,365.95,7.42,13.9
BRAP4.SA,23.34,NA,14.44,9.14,1.37,19.54,1.38,0.00,6.78,-0.46,NA
BRDT3.SA,28.33,NA,0.36,13.32,2.51,0.30,0.94,1.05,8.88,4.31,-13.2


In [48]:
ranking

,Trailing P/E,PEG Ratio (5 yr expected) 1,Enterprise Value/Revenue 3,Enterprise Value/EBITDA 6,Price/Book (mrq),Price/Sales (ttm),Beta (5Y Monthly),Profit Margin,Return on Equity (ttm),Return on Assets (ttm),Quarterly Revenue Growth (yoy)
ABEV3.SA,41,2,42,33,44,53,14,19,29,11,15
AZUL4.SA,24,9,43,61,71,31,51,70,67,68,67
B3SA3.SA,44,9,69,45,51,70,37,4,20,15,6
BBAS3.SA,4,9,72,61,6,23,48,11,21,57,43
BBDC3.SA,10,5,72,61,18,35,65,71,67,70,70
BBDC4.SA,11,4,72,61,22,38,65,71,67,70,70
BBSE3.SA,17,9,71,61,64,71,29,3,4,1,37
BEEF3.SA,1,9,6,10,51,4,18,40,1,13,31
BRAP4.SA,33,9,70,27,16,72,49,56,42,64,70
BRDT3.SA,42,9,1,37,40,7,29,53,36,30,58
